**Imports**

In [2]:
import json
import pandas as pd
import re
from pyarabic.araby import tokenize
from pyarabic.araby import normalize_ligature
from pyarabic.araby import tokenize
import tashaphyne.arabic_const as arabconst
with open("D:/Knowledge/NU/6th_Year/SPRI22/CSCI322_Data_Analysis/Project/result1.json",encoding="utf8") as f:
    data=json.load(f)
    
females_data={}
males_data={}

**Making Dataframe From the Telegram Messages**

In [3]:
def collect_data(form,dic,recNo):
    dic[recNo]={}
    for formLine in form:
        if(formLine=='' or formLine==' '):
            continue
        else:
            formLine=formLine.split('    ')
            formLine=list(filter(None,formLine))
            for record in formLine:
                record=record.split(':',1)
                if(len(record)<=1):
                    continue
                else:
                    record[0]=record[0].strip()
                    # Females Columns Cleaning (start)
                    if(record[0]=='لون البشرة.1'):
                        record[0]="color"
                    if(record[0]=='الوالدة' or record[0]=='وثلاثة في حضانة امهم' or record[0]=='- الوالده' or record[0]=='الوالده' or record[0]=='الام' or record[0]=='والدتي' or record[0]=='أمي'):
                        continue
                    if(record[0]=='hلتيسير الذي يمكن ان تقدمه لاتمام عملية الزواج' or record[0]=='ما هو شكل التيسير المقدم لاتمام عملية الزواج' or record[0]=='شكل التيسير المقدم' or record[0]=='شكل التيسير المقدم لاتمام الزواج'or record[0]=='شكل التيسير المقدم لاتمام عملية الزواج' or record[0]=='التيسير الذي يمكن ان تقدمه لاتمام عملية الزواج' or record[0]== 'لتيسير الذي يمكن ان تقدمه لاتمام عملية الزواج' or record[0]=='ما هو شكل التيسير الذي يمكن ان تقدميه لاتمام عملية الزواج'):
                        record[0]='التيسير المقدم لاتمام عملية الزواج'
                    if(record[0]=='اضافات ضرورية' or record[0]=='اضافات أخرى' or record[0]=='اضافات اخرى' or record[0]=='اضافات ضرورية.1' or record[0]=='هل تريدين اضافة شئ ترينه ضروريا؟' or record[0]=='هل تريدين إضافة شئ ترينه ضرورياً' or record[0]=='اضافت ضرورية' or record[0]=='أضافات ضرورية'):
                        record[0]='إضافات ضرورية'
                    if(record[0]=="الحالة الاجتماعية.1" or record[0]=='الحالة الاجتماعية لها' or record[0]=='حالة الأخت' or (record[0]=="الحالة الاجتماعية" and record[0] in dic[recNo].keys())):
                        record[0]='الحالة الإجتماعية المناسبة'
                    if(record[0]=='عدد الأولاد وأعمارهم وحضانتهم' or record[0]=='عدد الأولاد وأعماره وحضانتهم' or record[0]=='عدد الاولاد وحضانتهم'):
                        record[0]='عدد الأولاد وأعمارهم وحضانتهم إذا وجد'
                    if(record[0]=='الوظيفة او العمل ان وجد'):
                        record[0]='الوظيفة او العمل'
                    if(record[0]=='هل تحافظين علي السنن القولیة والفعلیة؟' or record[0]=='الخفاظ علي السنن القولیة والفعلیة' or record[0]=='الحفاظ السنن القولیة والفعلیة' or record[0]=='الحفاظ علي السنن القولیة والفعلیة'):
                        record[0]='الحفاظ على السنن القولیة والفعلیة'
                    if(record[0]=='هل تحافظين علي الصلاة في وقتها ؟'):
                        record[0]='الحفاظ علي الصلاة في وقتها'
                    if(record[0]=='حفظ القران الكريم' or record[0]=='مقدرا حفظ القران'):
                        record[0]='مقدار حفظ القران'
                    if(record[0]=='السن المناسب لكِ' or record[0]=='السن المناسب لها' or record[0]=='السن المناسب له'):
                        record[0]="السن المناسب"
                    if(record[0]=='المؤهل الذي تفضلينه' or record[0]=='المؤهل الذي تفضله' or record[0] =="المؤهل الذي يفضله"):
                        record[0]="المؤهل المفضل"
                    #end
                    # Males Columns Cleaning (start)
                    if(record[0]=='هل تدخن السجائر او غيرها' or record[0]=='تدخين السجائر'):
                        record[0]="تدخين السجائر او غيرها"
                    if(record[0]=='قيمة ونوع مهر الفتاة الذي يقدر عليه'):
                        record[0]='قيمة ونوع المهر'
                    if(record[0]=='ركود'):
                        record[0]="كود"
                    #end
                    dic[recNo][record[0]]=record[1].strip()

In [4]:
def getForm(text):
    global femalesRecord
    global malesRecord
    lines=text.split('\n')
    for line in lines:
        splits=line.split(':')
        for split in splits:
            if('كود' in split):
                form=text.split('\n')
                if('نساء' in form[0]):
                    femalesRecord+=1
                    collect_data(form,females_data,femalesRecord)
                elif('رجال' in form[0]):
                    malesRecord+=1
                    collect_data(form,males_data,malesRecord)                

In [5]:
femalesRecord=0
malesRecord=0
for message in data['messages']:
    if (message['type']=='message'):
        if type(message['text'])==list:
            i=0
            for txt in message['text']:
                if(type(txt)==dict):
                    message['text'][i]=txt['text']
                    i+=1
                else:
                    i+=1
            newTxt=''.join(message['text'])
            getForm(newTxt)
        else:
            getForm(message['text'])
females_df=pd.DataFrame.from_dict(females_data,orient='index')
males_df=pd.DataFrame.from_dict(males_data,orient='index')

**Changing Columns Names**

In [701]:
females_df.columns=['code','age','weight','height','color','nationality','education','job','status','children','parents_jobs','family_address','area_level','dress','maintain_prayer','mem_quraan','maintain_sunnah','accept_age','accept_weight','accept_height','accept_color','accept_education','accept_status','engage_period','facilit','additions','accept_resid','accept_address','facilitations']
males_df.columns=['code','age','weight','height','color','nationality','education','job','status','children','parents_jobs','family_address','area_level','future_resid','maintain_prayer_time','maintain_prayer_masjid','mem_quraan','maintain_sunnah','smoking','additions','accept_age','accept_weight','accept_height','accept_color','accept_education','accept_status','accept_dress','engage_period','mahr','activities']

**Normalizing Arabic Data**

In [702]:
def strip_tashkeel(text): 
    return arabconst.HARAKAT_PAT.sub('', text)

def strip_tatweel(text): 
    return re.sub(u'[%s]' % arabconst.TATWEEL,    '', text) 

def normalize_hamza(text): 
    text = arabconst.ALEFAT_PAT.sub(arabconst.ALEF, text) 
    return arabconst.HAMZAT_PAT.sub(arabconst.HAMZA, text)

def normalize_lamalef(text): 
    return arabconst.LAMALEFAT_PAT.sub( u'%s%s'%(arabconst.LAM, arabconst.ALEF), text) 

def normalize_spellerrors(text): 
    text = re.sub(u'[%s]' % arabconst.TEH_MARBUTA,    arabconst.HEH, text) 
    return re.sub(u'[%s]' % arabconst.ALEF_MAKSURA,    arabconst.YEH, text) 

def clean_str(text):
    search =[',', '(',')','_','+','"','#',' ','/',':','،','أ','.','آ']
    replace=[' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','ا',' ','ا']
    
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    return text
    
def lam(text):
    comment_words = ''
    val = str(text)
    # split the value
    tokens = val.split()
    # Converts each token into lowercase
    for i in range(len(tokens)):
        if tokens[i].startswith(u'ال'):
            tokens[i] = tokens[i].replace('ال', ' ')
    comment_words += " ".join(tokens)+" "
    return comment_words

def normalize_searchtext(text):
    text = lam(text) 
    text = clean_str(text)
    text = strip_tashkeel(text) 
    text = strip_tatweel(text) 
    text = normalize_lamalef(text) 
    text = normalize_spellerrors(text) 
    text = text.strip()
    return text 

**Males Education Column Cleaning**

In [703]:
males_df['education']=males_df['education'].apply(lambda x: normalize_searchtext(x))

In [704]:
def encode_education_males(text):
    search = ['جامعي' ,'فوق','متوسط']
    if search[0] in text and search[1] in text:
        text = 'فوق جامعي'
    elif search[0] in text:
       text = search[0]
    else:
        text = search[2]
    return text

In [705]:
males_df['education']=males_df['education'].apply(lambda x: encode_education_males(x))

**Males Color Column Cleaning**

In [706]:
males_df['color']=males_df['color'].apply(lambda x: normalize_searchtext(x))

In [707]:
def color_clean_males(text):
    search = ['بيض','بياض', 'خمري', 'قمحي','سمراء']
    replace = ['1','1','2','3','4']
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    return text

In [708]:
def encode_color_males(text):
    text = color_clean_males(text)
    if '4' in text:
        return 'اسمر'
    elif '3' in text:
        return 'قمحي'
    elif '2' in text:
        return'خمري'
    elif '1' in text:
        return 'ابيض'
    else:
        return 'قمحي'

In [709]:
males_df['color']=males_df['color'].apply(lambda x: encode_color_males(x))

**Males Nationality Column Cleaning**

In [710]:
males_df['nationality']=males_df['nationality'].apply(lambda x: normalize_searchtext(x))

In [711]:
def nationality_clean_males(text):
    if "مصري" in text:
        return "مصري"
    else:
        return text

In [712]:
males_df['nationality']=males_df['nationality'].apply(lambda x: nationality_clean_males(x))

**Males Status Column Cleaning**

In [713]:
males_df['status']=males_df['status'].apply(lambda x: normalize_searchtext(x))

In [714]:
def status_clean_males(text):
    if "متزوج" in text:
        return "متزوج"
    else:
        return text

In [715]:
males_df['status']=males_df['status'].apply(lambda x: status_clean_males(x))

**Males area_level Column Cleaning**

In [716]:
males_df['area_level']=males_df['area_level'].apply(lambda x: normalize_searchtext(x))

**Males accept_color Column Cleaning**

In [717]:
males_df['accept_color']=males_df['accept_color'].apply(lambda x: normalize_searchtext(x))

In [718]:
def accept_color_clean_males(text):
    search = ['بيض', 'خمري', 'قمحي','سمراء','اي']
    replace = ['1','2','3','4','5']
    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])
    return text

In [719]:
def encode_accept_color_males(text):
    text = color_clean_males(text)
    encode=''
    encode1='0'
    encode2='0'
    encode3='0'
    encode4='0'
    if '5' in text:
        encode1='1'
        encode2='1'
        encode3='1'
        encode4='1'
    else:
        if '4' in text:
            encode4='1'
        if '3' in text:
            encode3= '1'
        if '2' in text:
            encode2= '1'
        if '1' in text:
            encode1= '1'
        if not(('4' in text) or ('3' in text) or ('2' in text) or ('1' in text)):
            encode1='1'
            encode2='1'
            encode3='1'
            encode4='1'
    encode=encode1+encode2+encode3+encode4
    return encode

In [720]:
males_df['accept_color']=males_df['accept_color'].apply(lambda x: encode_accept_color_males(x))

**Males accept_education Column Cleaning**

In [721]:
males_df['accept_education']=males_df['accept_education'].apply(lambda x: normalize_searchtext(x))

In [722]:
def accept_education_clean_males(text):
    search=['جامعي','فوق','متوسط']
    if search[0] in text and search[1] in text:
        return 'فوق جامعي'
    elif search[0] in text:
        return search[0]
    else:
        return search[2]

In [723]:
males_df['accept_education']=males_df['accept_education'].apply(lambda x: accept_education_clean_males(x))

**Males accept_status Column Cleaning**

In [733]:
males_df['accept_status']=males_df['accept_status'].apply(lambda x: normalize_searchtext(x))

In [7]:
females_df

,كود,السن,الوزن,الطول,لون البشرة,الجنسية,المؤهل الدراسي,الوظيفة او العمل,الحالة الاجتماعية,عدد الأولاد وأعمارهم وحضانتهم إذا وجد,...,الطول المناسب,color,المؤهل المفضل,الحالة الإجتماعية المناسبة,فترة الخطوبة,التيسير المقدم لاتمام عملية الزواج,إضافات ضرورية,نوع السكن المقبول,مكان السكن المقبول,التيسير المقدم لاتمام عملية الزواج
1,2 نساء,35.0,90,150,بين القمحيه والسمراء,مصرية,"جامعي, بكالريوس تجارة",محاسبة,#انسة,لا يوجد,...,160,اي مما سبق,جامعي,مطلق/ أرمل بدون أولاد,3 شهور,تاثيث بيت الزوجيه,NaN,NaN,NaN,NaN
2,3 نساء,35.0,50,157,بيضاء,مصرية,متوسط,سكرتارية عيادات,#انسة,NaN,...,إن يكون مناسب,اي مما سبق,لا يهم,لم يسبق له الزواج,سنة ليس اكثر,كما قال الله و رسوله. و بالتيسير لا التعسيير,إن يكون شاب ع خلق و دين,NaN,NaN,NaN
3,4 نساء,44.0,80,170,قمحية,مصرية,"فوق الجامعي, دكتوراة وماجستير كليه علوم",دكتورة في مجال البحث العلمي,#انسة,NaN,...,١٧٠-١٩٠,"بيضاء, قمحية","فوق جامعي, جامعي","لم يسبق له الزواج, مطلق/ أرمل بدون أولاد",6 شهور,اساهم في تأسيس بيت الزوجيه بقدر المستطاع ولم ا...,NaN,NaN,NaN,NaN
4,6 نساء,25.0,68,160,قمحية,مصرية,جامعي,لا اعمل,#انسة,NaN,...,اكثر من 170,"بيضاء, قمحية","فوق جامعي, جامعي",لم يسبق له الزواج,3 شهور,التيسير ف متطلبات الزواج,NaN,NaN,NaN,NaN
5,8 نساء,28.0,فوق المتوسط,165,"بيضاء, قمحية",مصرية,بكالريوس تجارة شعبة محاسبة,لا أعمل,#انسة,NaN,...,١٧٥ مثلاً,NaN,"فوق جامعي, جامعي",لم يسبق له الزواج,6 شهور,مرجع الأمر لله إجمالاً أما القرار فبيد أبي,لا يوجد,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388,446 نساء,26,65.0,165.0,قمحية,مصرية,"جامعي, كليه الهندسه",مهندسه,#انسة,NaN,...,أطول من 169 سم,اي مما سبق,"فوق جامعي, جامعي",لم يسبق له الزواج,سنة ليس اكثر,يوجد تيسير في حاله القبول,أن يكون متدين و محافظ علي الصلوات الخمس في أوق...,"ايجار, ملك",المحافظة التي اسكن بها وما يجاورها,NaN
389,447 نساء,36,70.0,165.0,قمحية,مصرية,فوق الجامعي,حكومي,#انسة,NaN,...,لا يكون قصير,اي مما سبق,"فوق جامعي, جامعي","لم يسبق له الزواج, مطلق/ أرمل بدون أولاد",سنة ليس اكثر,حسب الاتفاق,إنسان تقي نقي يتقي الله و يخشى اليوم الآخر,"ايجار, ملك",اي محافظة,NaN
390,448 نساء,٢٨.٥,79.0,155.0,قمحية,مصرية,فوق الجامعي,لا يوجد,#انسة,NaN,...,165.0,اي مما سبق,"فوق جامعي, جامعي",لم يسبق له الزواج,سنة ليس اكثر,بالاتفاق,لا يوجد,لايهم,المحافظة التي اسكن بها وما يجاورها,NaN
391,449 نساء,32,65.0,165.0,بيضاء,مصرية,فوق الجامعي,مديره تصدير,#انسة,NaN,...,الطول المناسب لي,اي مما سبق,جامعي,لم يسبق له الزواج,سنة ليس اكثر,القدر المناسب للزواج,زوج صالح يتقي الله فيه,ملك,المحافظة التي اسكن بها وما يجاورها,NaN
